# Demo Notebook how to run models on static mouse datasets

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [2]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True

name = "test"
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [3]:
import torch
import numpy as np
import pickle 
import pandas as pd
from collections import OrderedDict, Iterable

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder

from nnsysident.tables.experiments import *
from nnsysident.datasets.mouse_loaders import static_shared_loaders
from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.datasets.mouse_loaders import static_loader

<ipython-input-3-71e7f2af38a6>:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import OrderedDict, Iterable


Connecting konstantin@sinzlab.chlkmukhxp6i.eu-central-1.rds.amazonaws.com:3306


# Get Dataloader

In [6]:
# change path here
paths = ['data/static22564-2-12-preproc0.zip',
                     'data/static22564-2-13-preproc0.zip',
                     'data/static22564-3-8-preproc0.zip',
                     'data/static22564-3-12-preproc0.zip']

dataset_fn = 'nnsysident.datasets.mouse_loaders.static_shared_loaders'
dataset_config = dict(
    paths=paths,
    batch_size=64,
    multi_match_n=3625,
    image_n=4399,
    seed=1
)
dataloaders = builder.get_data(dataset_fn, dataset_config)

data/static22564-2-12-preproc0 exists already. Not unpacking data/static22564-2-12-preproc0.zip
data/static22564-2-13-preproc0 exists already. Not unpacking data/static22564-2-13-preproc0.zip
data/static22564-3-8-preproc0 exists already. Not unpacking data/static22564-3-8-preproc0.zip
data/static22564-3-12-preproc0 exists already. Not unpacking data/static22564-3-12-preproc0.zip
data/static22564-2-12-preproc0 exists already. Not unpacking data/static22564-2-12-preproc0.zip
data/static22564-2-13-preproc0 exists already. Not unpacking data/static22564-2-13-preproc0.zip
data/static22564-3-8-preproc0 exists already. Not unpacking data/static22564-3-8-preproc0.zip
data/static22564-3-12-preproc0 exists already. Not unpacking data/static22564-3-12-preproc0.zip


# Get Model

### The old gaussian readout

In [ ]:
model_fn = 'nnvision.models.se_core_gauss_readout'
model_config = {
   'pad_input': False,
   'stack': -1,
   'layers':4,
   'input_kern': 9,
   'gamma_input': 20,
   'gamma_readout': 0.012,
   'hidden_dilation': 1,
   'hidden_kern': 7,
   'hidden_channels': 64,
    'depth_separable': True,
    
}
model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1000)

### Spatial Transformer

In [ ]:
model_fn = 'nnvision.models.se_core_spatialXfeature_readout'
model_config = {
   'pad_input': False,
   'stack': -1,
   'layers':4,
   'input_kern': 9,
   'gamma_input': 20,
   'gamma_readout': 0.005,
   'hidden_dilation': 1,
   'hidden_kern': 7,
   'hidden_channels': 64,
    'init_noise': 1e-3,
    'depth_separable': True,
    
}
model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1000)

### The New gaussian readout: change gauss_type for the different modes

In [8]:
model_fn = 'nnsysident.models.models.se2d_fullgaussian2d'

model_config = {'share_features': True,
 'init_mu_range': 0.55,
 'init_sigma': 0.4,
 'input_kern': 15,
 'hidden_kern': 13,
 'gamma_input': 1.0,
 'gamma_readout': 2.117604964706911,
 'grid_mean_predictor': {'type': 'cortex',
  'input_dimensions': 2,
  'hidden_layers': 0,
  'hidden_features': 0,
  'final_tanh': False}}


model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)

/src/nnsysident/nnsysident/models/models.py:139: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  all_multi_unit_ids = set(np.hstack(shared_match_ids.values()))


In [ ]:
model_fn = 'nnsysident.models.models.se2d_deterministicgaussian2d'
model_config = {#'share_features': True
}
model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)

In [ ]:
model_fn = 'nnsysident.models.models.se2d_spatialxfeaturelinear'
model_config = {
}
model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)

In [ ]:
restricted_trained_model_table = (trained_model_table &
                                          "model_hash = '{}'".format(t_model_hash) &
                                          "dataset_hash = '{}'".format(t_dataset_hash) &
                                          "trainer_hash = '{}'".format(t_trainer_hash))
trained_model_entries = pd.DataFrame(restricted_trained_model_table.fetch())
trained_model_entry = trained_model_entries.loc[trained_model_entries['score'] == trained_model_entries['score'].max()]
state_dict = (restricted_trained_model_table * restricted_trained_model_table.ModelStorage & "seed = {}".format(int(trained_model_entry['seed']))).fetch1('model_state', download_path='models/')
core_dict = OrderedDict([(k, v) for k, v in torch.load(state_dict).items() if k[0:5] == 'core.'])
model.load_state_dict(core_dict, strict=False)

# Get Trainer

In [9]:
trainer_fn = 'nnsysident.training.trainers.standard_trainer'
trainer_config = dict()
trainer = builder.get_trainer(trainer_fn, trainer_config)

# Run Training

In [10]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=1)

Epoch 1: 100%|██████████| 276/276 [01:00<00:00,  4.55it/s]


[001|00/05] ---> 0.052468642592430115


Epoch 2: 100%|██████████| 276/276 [00:33<00:00,  8.16it/s]


[002|00/05] ---> 0.07284536957740784


Epoch 3: 100%|██████████| 276/276 [00:34<00:00,  8.01it/s]


...

[086|03/05] -/-> 0.3728507161140442


Epoch 87: 100%|██████████| 276/276 [00:34<00:00,  7.91it/s]


[087|04/05] -/-> 0.3697288930416107


Epoch 88: 100%|██████████| 276/276 [00:35<00:00,  7.88it/s]


[088|05/05] -/-> 0.3739948272705078
Restoring best model after lr decay! 0.373995 ---> 0.374988
Restoring best model! 0.374988 ---> 0.374988
